In [18]:
import pandas as pd 
import numpy as np 
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [19]:
train_df = pd.read_csv("train.csv")[["ID", "timestamp", "supply(kg)", "price(원/kg)"]] 

In [20]:
train_df["item_id"] = train_df.ID.str[0:6] 

In [21]:
data = TimeSeriesDataFrame(train_df.drop(columns=["ID"])) 

data.head()

supply(kg)  price(원/kg)
item_id timestamp                          
TG_A_J  2019-01-01         0.0          0.0
        2019-01-02         0.0          0.0
        2019-01-03     60601.0       1728.0
        2019-01-04     25000.0       1408.0
        2019-01-05     32352.0       1250.0

In [22]:
predictor = TimeSeriesPredictor(
    prediction_length=28, 
    target="price(원/kg)", 
    eval_metric="RMSE" 
) 

predictor.fit(data,
              random_seed=42)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231113_052917/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)']

Starting training. Start time is 202

In [23]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	2.06    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 2.11 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [24]:
pred = predictor.predict(data, random_seed=42) 

Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [25]:
pred.head()

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3301.639500  1586.520724  2187.282024  2595.555044   
        2023-03-05   590.913883 -1357.727693  -689.942047  -208.152723   
        2023-03-06  3122.654790   278.562916  1294.456780  1979.468450   
        2023-03-07  3414.310521   209.623718  1323.730200  2160.976267   
        2023-03-08  3301.728344  -171.812420  1079.410574  1941.028047   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2955.979059  3272.499293  3611.822235  3973.537287   
        2023-03-05   201.772270   583.851928   968.612013  1378.965727   
        2023-03-06  2586.093867  3127.782085  3702.951233  4330.210204   
        2023-03-07  2817.866899  3436.391044  4053.273484  4726.519563   
        2023-03-08  2660.993779  3327.277798  3980.459797  4708.867402   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4397.753906  5011.822713  
        2023-03-05  1859.398811  2525.347335  
        2023-03-06  5010.268353  5977.537585  
        2023-03-07  5490.898722  6697.052141  
        2023-03-08  5531.296740  6843.241892

In [26]:
submission = pd.read_csv("sample_submission.csv") 
submission["answer"] = pred.reset_index()["mean"] 
submission.loc[submission['answer'] < 0.0, 'answer'] = 0.0  
submission.to_csv("autogluon_dacon_basic.csv", index=False) 

In [27]:
submission

,ID,answer
0,TG_A_J_20230304,3301.639500
1,TG_A_J_20230305,590.913883
2,TG_A_J_20230306,3122.654790
3,TG_A_J_20230307,3414.310521
4,TG_A_J_20230308,3301.728344
...,...,...
1087,RD_F_J_20230327,531.722071
1088,RD_F_J_20230328,525.491683
1089,RD_F_J_20230329,545.552595
1090,RD_F_J_20230330,504.625521
